In [1]:
import os
import os.path
import pickle
import time
import shelve

import chainer
from chainer import cuda
from chainer import serializers
import chainer.optimizers as O
import numpy as np

from lda2vec import utils
from lda2vec import prepare_topics, print_top_words_per_topic, topic_coherence
from lda2vec import LDA2Vec

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# gpu_id = int(os.getenv('CUDA_GPU', 0))
# cuda.get_device(gpu_id).use()
# print("Using GPU:" + str(gpu_id))

In [3]:
#data_dir = os.getenv('data_dir', '../data/')
fn_vocab = 'vocab.pkl'
fn_corpus = 'corpus.pkl'
fn_flatnd = 'flattened.npy'
fn_docids = 'doc_ids.npy'
fn_vectors = 'vectors.npy'
vocab = pickle.load(open(fn_vocab, 'rb'))
corpus = pickle.load(open(fn_corpus, 'rb'))
flattened = np.load(fn_flatnd)
doc_ids = np.load(fn_docids)
vectors = np.load(fn_vectors)

In [15]:
# Model Parameters
# Number of documents
n_docs = doc_ids.max() + 1
# Number of unique words in the vocabulary
n_vocab = flattened.max() + 1
# 'Strength' of the dircihlet prior; 200.0 seems to work well
clambda = 200.0
# Number of topics to fit
n_topics = int(os.getenv('n_topics', 30))
batchsize = 4096
# Power for neg sampling
power = float(os.getenv('power', 0.75))
# Intialize with pretrained word vectors
pretrained = bool(int(os.getenv('pretrained', True)))
# Sampling temperature
temperature = float(os.getenv('temperature', 1.0))
# Number of dimensions in a single word vector
n_units = int(os.getenv('n_units', 300))
# Get the string representation for every compact key
words = corpus.word_list(vocab)[:n_vocab]
# How many tokens are in each document
doc_idx, lengths = np.unique(doc_ids, return_counts=True)
doc_lengths = np.zeros(doc_ids.max() + 1, dtype='int32')
doc_lengths[doc_idx] = lengths
# Count all token frequencies
tok_idx, freq = np.unique(flattened, return_counts=True)
term_frequency = np.zeros(n_vocab, dtype='int32')
term_frequency[tok_idx] = freq

In [16]:
# for key in sorted(locals().keys()):
#     val = locals()[key]
#     if len(str(val)) < 100 and '<' not in str(val):
#         print(key, val)

# training the model

In [17]:
model = LDA2Vec(n_documents=n_docs, n_document_topics=n_topics,
                n_units=n_units, n_vocab=n_vocab, counts=term_frequency,
                n_samples=20, power=power, temperature=temperature)

(39, 300)
[[0.37454012 0.95071431 0.73199394 ... 0.21582103 0.62289048 0.08534746]
 [0.05168172 0.53135463 0.54063512 ... 0.17231987 0.19228902 0.04086862]
 [0.16893506 0.27859034 0.17701048 ... 0.89633582 0.01300192 0.08550853]
 ...
 [0.35121702 0.83724    0.06679708 ... 0.82626915 0.7313451  0.98933329]
 [0.27254765 0.8749895  0.76662267 ... 0.68798402 0.3228145  0.54548815]
 [0.14727381 0.4525645  0.88119734 ... 0.92717484 0.90138686 0.08987828]]


In [18]:
# if os.path.exists('lda2vec.hdf5'):
#     print("Reloading from saved")
#     serializers.load_hdf5("lda2vec.hdf5", model)
    
if pretrained:
    model.sampler.W.data[:, :] = vectors[:n_vocab, :]

In [19]:
# model.to_gpu()
optimizer = O.Adam()
optimizer.setup(model)
clip = chainer.optimizer.GradientClipping(5.0)
optimizer.add_hook(clip)

In [20]:
j = 0
epoch = 0
fraction = batchsize * 1.0 / flattened.shape[0]
progress = shelve.open('progress.shelve')

In [21]:
for epoch in range(1):
    data = prepare_topics(cuda.to_cpu(model.mixture.weights.W.data).copy(),
                          cuda.to_cpu(model.mixture.factors.W.data).copy(),
                          cuda.to_cpu(model.sampler.W.data).copy(),
                          words)
    top_words = print_top_words_per_topic(data)
    if j % 100 == 0 and j > 100:
        coherence = topic_coherence(top_words)
        for j in range(n_topics):
            print(j, coherence[(j, 'cv')])
        kw = dict(top_words=top_words, coherence=coherence, epoch=epoch)
        progress[str(epoch)] = pickle.dumps(kw)
    data['doc_lengths'] = doc_lengths
    data['term_frequency'] = term_frequency
    np.savez('topics.pyldavis', **data)
    print(epoch)
    for d, f in utils.chunks(batchsize, doc_ids, flattened):
        t0 = time.time()
        model.cleargrads()
        #optimizer.use_cleargrads(use=False)
        l = model.fit_partial(d.copy(), f.copy())
        print("after partial fitting:", l)
        prior = model.prior()
        loss = prior * fraction
        loss.backward()
        optimizer.update()
        msg = ("J:{j:05d} E:{epoch:05d} L:{loss:1.3e} "
               "P:{prior:1.3e} R:{rate:1.3e}")
        prior.to_cpu()
        loss.to_cpu()
        t1 = time.time()
        dt = t1 - t0
        rate = batchsize / dt
        logs = dict(loss=float(l), epoch=epoch, j=j,
                    prior=float(prior.data), rate=rate)
        print(msg.format(**logs))
        j += 1
    serializers.save_hdf5("lda2vec.hdf5", model)

Top words in topic 0 plentiful high service price atmosphere value flavorful fresh food appropriate
Top words in topic 1 sushi flavorful plentiful relaxed friendly casual good place like delicious
Top words in topic 2 sushi menu great appropriate going restaurant delicious like friendly view
Top words in topic 3 good reasonably fresh delicious high nice flavorful relaxed dish sushi
Top words in topic 4 casual place atmosphere friendly great plentiful always good menu relaxed
Top words in topic 5 nice restaurant menu casual sushi would dish meal absolutely plentiful
Top words in topic 6 food relaxed atmosphere restaurant casual fresh high staff <SKIP> good
Top words in topic 7 plentiful price sushi view service flavorful food out_of_vocabulary staff best
Top words in topic 8 service food price out_of_vocabulary value out_of_vocabulary appropriate place view restaurant
Top words in topic 9 fresh price going would amazing staff high great absolutely service
Top words in topic 10 fresh foo

In [22]:
all_topics = []
for row in top_words:
    for word in row:
        all_topics.append(word)
print(sorted(list(dict.fromkeys(all_topics))))
print(len(sorted(list(dict.fromkeys(all_topics)))))

['<SKIP>', 'absolutely', 'always', 'amazing', 'appropriate', 'atmosphere', 'best', 'casual', 'delicious', 'dish', 'excellent', 'flavorful', 'food', 'fresh', 'friendly', 'going', 'good', 'great', 'high', 'like', 'meal', 'menu', 'nice', 'out_of_vocabulary', 'pizza', 'place', 'plentiful', 'price', 'really', 'reasonably', 'relaxed', 'restaurant', 'service', 'staff', 'sushi', 'value', 'view', 'would']
38


In [23]:
import pandas as pd

df = pd.read_csv('../res_mul_all.csv')
df.head()

,id,reviewID,sentenceID,review,category,polarity,entity,preprocessed_sentence,type_sentence
0,0,RL#3,RL#3:1,I am not necessarily fanatical about this plac...,VALUE#PRICES,positive,VALUE,i am not necessarily fanatical about this plac...,compound_sentence
1,2,TR#2,TR#2:2,The high prices you're going to pay is for the...,VALUE#PRICES,negative,VALUE,the high prices you 're going to pay is for th...,complex_sentence
2,3,TR#2,TR#2:2,The high prices you're going to pay is for the...,VALUE#PRICES,negative,VALUE,the high prices you 're going to pay is for th...,complex_sentence
3,4,TR#2,TR#2:2,The high prices you're going to pay is for the...,VALUE#PRICES,negative,VALUE,the high prices you 're going to pay is for th...,complex_sentence
4,5,TR#2,TR#2:2,The high prices you're going to pay is for the...,VALUE#PRICES,negative,VALUE,the high prices you 're going to pay is for th...,complex_sentence


In [24]:
import pandas as pd

def aspect_topic(tipe, all_topics):
    sf = pd.DataFrame(columns=['id','review','category','term'])
    count = 0
    index = 0
    res = []
    for sentence in df['review']:
        lowercased = sentence.lower()
        term = []
        category = []
        for cat in df['category'][index].split(','):
            splitted = cat.split('#')
            if splitted[1] == 'PRICES':
                category.append('VALUE')
            else:
                category.append(splitted[0])
        id_name = df['id'][index]
        for topic in all_topics:
            if topic in lowercased:
                term.append(topic)
#         print(term)
        if len(term) == 0:
            print(lowercased)
            count += 1
        sf = sf.append({'id': id_name, 'review': sentence.strip().lower().replace('  ', ' '), 'category': '|'.join(category), 'term': '|'.join(term)}, ignore_index=True)
        index += 1
    print(count)
    sf.to_csv("lda2vec"+ tipe +".csv")
    sf.to_excel("lda2vec"+ tipe +".xlsx")

In [25]:
aspect_topic("-20",list(dict.fromkeys(all_topics)))

chow fun was dry; pork shu mai was more than usually greasy and had to share a table with loud and rude family. 
i had the duck breast special on my last visit and it was not incredible.
the only thing i moderately enjoyed was their grilled chicken special with edamame puree.
i had never had edamame pureed before but i thought it was innovative and tasty (could've used a bit more salt).
the lava cake dessert was terrible.
once you step into cosette, you're miraculously in a small, off-the-beaten path parisian bistro.
my wife had the fried shrimp which are huge and loved it.
the hostess is rude to the point of being offensive.
there was a small wait, but shorter than i expected.
first went here to enjoy their garden terrace.
took my mom for mother's day, and the maitre d' was pretty rude.
i loved everythig about it-especially the shows and actors.
the tuna and wasabe potatoes are bad.
the bagel was small.
salads were bad.
ingredients are organic which is a real plus for me.
we even had 